In [1]:
using Queryverse
using IndexedTables
using Statistics
using MLJ
using ShapML
using DataFrames

In [2]:
survey_data = Queryverse.load("multipleChoiceResponses.csv") |> 
    # Redundant first row
    @drop(1) |>
    DataFrame;

In [3]:
survey_data |> IndexedTables.table

Table with 23859 rows, 395 columns:
Columns:
#    colname                              type
────────────────────────────────────────────────
1    Time from Start to Finish (seconds)  String
2    Q1                                   String
3    Q1_OTHER_TEXT                        String
4    Q2                                   String
5    Q3                                   String
6    Q4                                   String
7    Q5                                   String
8    Q6                                   String
9    Q6_OTHER_TEXT                        String
10   Q7                                   String
11   Q7_OTHER_TEXT                        String
12   Q8                                   String
13   Q9                                   String
14   Q10                                  String
15   Q11_Part_1                           String
16   Q11_Part_2                           String
17   Q11_Part_3                           String
18   Q11_Part_4           

In [4]:
survey_data |> 
    describe

,variable,mean,min,median,max
,Symbol,Nothing,String,Nothing,String
1,Time from Start to Finish (seconds),,100,,9992
2,Q1,,Female,,Prefer to self-describe
3,Q1_OTHER_TEXT,,-1,,9
4,Q2,,18-21,,80+
5,Q3,,Argentina,,Viet Nam
6,Q4,,,,Some college/university study without earning a bachelor’s degree
7,Q5,,,,"Social sciences (anthropology, psychology, sociology, etc.)"
8,Q6,,,,Student
9,Q6_OTHER_TEXT,,-1,,99


In [5]:
survey_data_clean = survey_data |>
    @rename(:Q1 => :gender,
            :Q2 => :age,
            :Q3 => :country,
            :Q4 => :education_level,
            :Q5 => :education_major,
            :Q6 => :job_role,
            :Q7 => :job_industry,
            :Q8 => :job_experience_years,
            :Q9 => :job_salary,
            :Q17 => :job_most_used_language,
            :Q23 => :job_percent_time_coding,
            :Q26 => :job_is_data_scientist) |>
    @select(-occursin("seconds"), 
            -occursin("Q")) |>
    # Could be done with less typing by using joins
    @mutate(age = 
             _.age == "18-21" ? 19.5 :
             _.age == "22-24" ? 23.0 :
             _.age == "25-29" ? 27.0 :
             _.age == "30-34" ? 32.0 :
             _.age == "35-39" ? 37.0 :
             _.age == "40-44" ? 42.0 :
             _.age == "45-49" ? 47.0 :
             _.age == "50-54" ? 52.0 :
             _.age == "55-59" ? 57.0 :
             _.age == "60-69" ? 64.5 :
             _.age == "70-79" ? 74.5 :
             _.age == "80+" ? 90 : NA,
            job_experience_years = 
            _.job_experience_years == "0-1" ? 0.5 :
            _.job_experience_years == "1-2" ? 1.5 :
            _.job_experience_years == "2-3" ? 2.5 :
            _.job_experience_years == "3-4" ? 3.5 :
            _.job_experience_years == "4-5" ? 4.5 :
            _.job_experience_years == "5-10" ? 7.5 :
            _.job_experience_years == "10-15" ? 12.5 :
            _.job_experience_years == "15-20" ? 17.5 :
            _.job_experience_years == "20-25" ? 22.5 :
            _.job_experience_years == "25-30" ? 27.5 :
            _.job_experience_years == "30 +" ? 40 : NA,
            job_salary = 
            _.job_salary == "0-10,000" ? 5_000 :
            _.job_salary == "10-20,000" ? 15_000 :
            _.job_salary == "20-30,000" ? 25_000 :
            _.job_salary == "30-40,000" ? 35_000 :
            _.job_salary == "40-50,000" ? 45_000 :
            _.job_salary == "50-60,000" ? 55_000 :
            _.job_salary == "60-70,000" ? 65_000 :
            _.job_salary == "70-80,000" ? 75_000 :
            _.job_salary == "80-90,000" ? 85_000 :
            _.job_salary == "90-100,000" ? 95_000 :
            _.job_salary == "100-125,000" ? 112_500 :
            _.job_salary == "125-150,000" ? 137_500 :
            _.job_salary == "150-200,000" ? 175_000 :
            _.job_salary == "200-250,000" ? 225_000 :
            _.job_salary == "250-300,000" ? 275_000 :
            _.job_salary == "300-400,000" ? 350_000 :
            _.job_salary == "400-500,000" ? 450_000 :
            _.job_salary == "500,000+" ? 500_000 : NA,
            job_percent_time_coding =
            _.job_percent_time_coding == "0% of my time" ? 0 :
            _.job_percent_time_coding == "1% to 25% of my time" ? 13 :
            _.job_percent_time_coding == "25% to 49% of my time" ? 37 :
            _.job_percent_time_coding == "50% to 74% of my time" ? 62 :
            _.job_percent_time_coding == "75% to 99% of my time" ? 87 :
            _.job_percent_time_coding == "100% of my time" ? 100 : NA) |>
  @dropna() |>
  @mutate(age = Float64(_.age),
          job_experience_years = Float64(_.job_experience_years),
          job_salary = Float64(_.job_salary),
          job_percent_time_coding = Float64(_.job_percent_time_coding) / 100) |>
  DataFrame

,gender,age,country,education_level
,Any,Any,Any,Any
1,Male,32.0,Indonesia,Bachelor’s degree
2,Female,32.0,United States of America,Master’s degree
3,Male,23.0,India,Master’s degree
4,Male,37.0,Chile,Doctoral degree
5,Male,19.5,India,Master’s degree
6,Male,32.0,Hungary,Master’s degree
7,Male,42.0,United States of America,Master’s degree
8,Male,27.0,United States of America,Bachelor’s degree
9,Female,27.0,Argentina,Bachelor’s degree


In [6]:
survey_data_clean |> 
    describe |> 
    @select(-:min, -:median, -:max)

variable,mean,nmissing,eltype
:gender,nothing,0,Any
:age,31.3967,0,Any
:country,nothing,0,Any
:education_level,nothing,0,Any
:education_major,nothing,0,Any
:job_role,nothing,0,Any
:job_industry,nothing,0,Any
:job_experience_years,4.99798,0,Any
:job_salary,51978.3,0,Any
:job_most_used_language,nothing,0,Any


In [7]:
survey_data_clean |> 
    describe |> 
    @select(:variable, :min, :median, :max)

variable,min,median,max
:gender,"""Female""",nothing,"""Prefer to self-describe"""
:age,19.5,27.0,90.0
:country,"""Argentina""",nothing,"""Viet Nam"""
:education_level,"""Bachelor’s degree""",nothing,"""Some college/university study without earning a bachelor’s degree"""
:education_major,"""""",nothing,"""Social sciences (anthropology, psychology, sociology, etc.)"""
:job_role,"""Business Analyst""",nothing,"""Student"""
:job_industry,"""Academics/Education""",nothing,"""Shipping/Transportation"""
:job_experience_years,0.5,2.5,40.0
:job_salary,5000.0,35000.0,500000.0
:job_most_used_language,"""""",nothing,"""Visual Basic/VBA"""


In [8]:
names(survey_data_clean)

12-element Vector{String}:
 "gender"
 "age"
 "country"
 "education_level"
 "education_major"
 "job_role"
 "job_industry"
 "job_experience_years"
 "job_salary"
 "job_most_used_language"
 "job_percent_time_coding"
 "job_is_data_scientist"

In [9]:
survey_data_clean |>
  size

(13399, 12)

In [10]:
# No NAs
survey_data_clean |> 
  @dropna() |> 
  DataFrame |> 
  size

(13399, 12)